In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
climate_type_dict = {'1':'BSk',
 '9': 'Dfb',
 '4': 'Cfa',
 '7': 'Csb',
 '8': 'Dfa',
 '3': 'BWk',
 '10': 'Dfc',
 '2': 'BWh',
 '6': 'Csa',
 '11': 'Dsb',
 '0': 'BSh',
 '5': 'Cfb',
 '12': 'Dsc',
 '13': 'Dwa',
 '14': 'Dwb'}

In [4]:
all_cols = add_train.columns.tolist()

In [5]:
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
train_df =  pd.read_csv('fe_v1_train.csv')
test_df  =  pd.read_csv('fe_v1_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [7]:
exclude_cols = ['index', 'startdate']
best_cols = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(best_cols))

Main features: 63


In [8]:
for c in all_cols:
    if 'contest-precip-14d' in c:
        print(c)

contest-precip-14d__precip


In [9]:
def calculate_season(month):
    if month == 12 or month == 1 or month == 2:
        return 4
    if month == 9 or month == 10 or month == 11:
        return 3
    if month == 7 or month == 8 or month == 6:
        return 2
    if month == 3 or month == 4 or month == 5:
        return 1

In [10]:
train_df['month'] = add_train['month']
test_df['month'] = add_test['month']

train_df['season'] = train_df['month'].apply(calculate_season)
test_df['season'] = test_df['month'].apply(calculate_season)

train_df['loc_group'] = add_train['loc_group']
test_df['loc_group'] = add_test['loc_group']

train_df['year'] = add_train['year']
test_df['year'] = add_test['year']

def map_climate_type(x):
    return climate_type_dict[str(x)]

train_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)
test_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)

precip_col = 'contest-precip-14d__precip'
wind_col = 'contest-wind-h500-14d__wind-hgt-500'

group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']


In [11]:
slp_col = 'contest-slp-14d__slp'

In [12]:
print(train_df[slp_col].min(), train_df[slp_col].max())
print(test_df[slp_col].min(), test_df[slp_col].max())

100379.21 103275.84
100593.22 102258.27


In [13]:
print(train_df[wind_col].min(), train_df[wind_col].max())
print(test_df[wind_col].min(), test_df[wind_col].max())

5289.25 5958.2
5504.59 5925.56


In [14]:
train_df[wind_col] = train_df[wind_col] - 5000
test_df[wind_col] = test_df[wind_col] - 5000

In [15]:
train_df[slp_col] = train_df[slp_col] - 100000
test_df[slp_col] = test_df[slp_col] - 100000

In [16]:
check_cols = [c for c in train_df.columns if c not in target and c not in exclude_cols]

df = pd.concat([train_df[check_cols], test_df[check_cols]], axis=0)
df.shape

(407088, 68)

In [17]:
def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

In [18]:
def assign_train_test_df(my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()

In [19]:
first_slp_df = create_df_by_group_and_agg(df, season_group_cols, "first", slp_col, "first_slp")
first_slp_df.head(3)

Nan


,loc_group,climate_type,year,season,first_slp
0,0,BSh,2014,3,1352.08
1,0,BSh,2014,4,2072.81
2,0,BSh,2015,1,1917.24


In [20]:
first_wind_df = create_df_by_group_and_agg(df, season_group_cols, "first", wind_col, "first_wind")
first_wind_df.head(3)

Nan


,loc_group,climate_type,year,season,first_wind
0,0,BSh,2014,3,899.66
1,0,BSh,2014,4,847.35
2,0,BSh,2015,1,804.47


In [21]:
first_slp_df_month = create_df_by_group_and_agg(df, group_cols, "first", slp_col, "first_slp_month")
first_slp_df_month.head(3)

Nan


,loc_group,climate_type,year,season,month,first_slp_month
0,0,BSh,2014,3,9,1352.08
1,0,BSh,2014,3,10,1254.96
2,0,BSh,2014,3,11,1952.19


In [22]:
df = df.merge(first_slp_df, how='left', on=season_group_cols)
df['diff_slp_first'] = df[slp_col] - df['first_slp']
assign_train_test_df('diff_slp_first')

In [23]:
df = df.merge(first_wind_df, how='left', on=season_group_cols)
df['diff_wind_first'] = df[wind_col] - df['first_wind']
assign_train_test_df('diff_wind_first')

In [29]:
features = best_cols + [
    'diff_slp_first',
    'diff_wind_first'
]

In [30]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_wind_v2.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:04,  1.12it/s]


(375734, 65) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 5.37ms	remaining: 26.8s
200:	learn: 1.1816269	test: 1.1877351	best: 1.1877351 (200)	total: 962ms	remaining: 23s
400:	learn: 0.9299113	test: 0.9392899	best: 0.9392899 (400)	total: 1.95s	remaining: 22.4s
600:	learn: 0.7944002	test: 0.8057680	best: 0.8057680 (600)	total: 2.86s	remaining: 21s
800:	learn: 0.7099965	test: 0.7231896	best: 0.7231896 (800)	total: 3.81s	remaining: 20s
1000:	learn: 0.6481700	test: 0.6623458	best: 0.6623458 (1000)	total: 4.78s	remaining: 19.1s
1200:	learn: 0.6014709	test: 0.6166821	best: 0.6166821 (1200)	total: 5.77s	remaining: 18.3s
1400:	learn: 0.5674466	test: 0.5836608	best: 0.5836608 (1400)	total: 6.8s	remaining: 17.5s
1600:	learn: 0.5377588	test: 0.5551651	best: 0.5551651 (1600)	total: 7.76s	remaining: 16.5s
1800:	learn: 0.5118741	test: 0.5301964	best: 0.5301964 (1800)	total: 8.71s	remaining: 15.5s
2000:	learn: 0.4908837	test: 0.5100865	best: 0.5100865 (2000)	total: 9

 20%|█████████████████▌                                                                      | 1/5 [00:34<02:19, 34.97s/it]

...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 5.64ms	remaining: 28.2s
200:	learn: 1.1689113	test: 1.1833073	best: 1.1833073 (200)	total: 932ms	remaining: 22.3s
400:	learn: 0.9261163	test: 0.9410892	best: 0.9410892 (400)	total: 1.9s	remaining: 21.8s
600:	learn: 0.7951683	test: 0.8098179	best: 0.8098179 (600)	total: 2.8s	remaining: 20.5s
800:	learn: 0.7076744	test: 0.7223040	best: 0.7223040 (800)	total: 3.72s	remaining: 19.5s
1000:	learn: 0.6464089	test: 0.6615705	best: 0.6615705 (1000)	total: 4.67s	remaining: 18.7s
1200:	learn: 0.6014900	test: 0.6176572	best: 0.6176572 (1200)	total: 5.59s	remaining: 17.7s
1400:	learn: 0.5654909	test: 0.5821292	best: 0.5821292 (1400)	total: 6.55s	remaining: 16.8s
1600:	learn: 0.5354246	test: 0.5532895	best: 0.5532895 (1600)	total: 7.42s	remaining: 15.8s
1800:	learn: 0.5098802	test: 0.5283577	best: 0.5283577 (1800)	total: 8.36s	remaining: 14.8s
2000:	learn: 0.4880848	test: 0.5072815	best: 0.5072815 (2000)	tot

 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [02:16<00:33, 33.36s/it]

Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 6.58ms	remaining: 32.9s
200:	learn: 1.1836220	test: 1.1894843	best: 1.1894843 (200)	total: 918ms	remaining: 21.9s
400:	learn: 0.9304308	test: 0.9396926	best: 0.9396926 (400)	total: 1.86s	remaining: 21.3s
600:	learn: 0.7988928	test: 0.8090954	best: 0.8090954 (600)	total: 2.83s	remaining: 20.7s
800:	learn: 0.7136536	test: 0.7255573	best: 0.7255573 (800)	total: 3.86s	remaining: 20.2s
1000:	learn: 0.6539405	test: 0.6668958	best: 0.6668958 (1000)	total: 4.81s	remaining: 19.2s
1200:	learn: 0.6089804	test: 0.6231424	best: 0.6231424 (1200)	total: 5.75s	remaining: 18.2s
1400:	learn: 0.5728738	test: 0.5881329	best: 0.5881329 (1400)	total: 6.7s	remaining: 17.2s
1600:	learn: 0.5432479	test: 0.5594801	best: 0.5594801 (1600)	total: 7.64s	remaining: 16.2s
1800:	learn: 0.5170494	test: 0.5343160	best: 0.5343160 (1800)	total: 8.6s	remaining: 15.3s
2000:	learn: 0.4949861	test: 0.5131007	best: 0.5131007 (2000)	tot

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 27.25it/s]


cos_sim with best submission: 0.9998429815389569
